In [ ]:
import os
import json
import pandas as pd
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from utils import detect_encoding, ip_in_subnet, my_font, generate_test_data  # 这些方法假设存在utils.py中
from rule import AnomalyRules
import gradio as gr
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

logging.basicConfig(
    level=logging.DEBUG,  # 设置为 DEBUG 级别以捕获调试信息
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]  # 输出到控制台
)

In [ ]:
class ModelTrainer:
    def __init__(self, config=None):
        """
        初始化 ModelTrainer，设置默认路径和规则文件。
        :param config: 配置字典，用于覆盖默认参数
        """
        default_config = {
            "data_dir": "data",
            "conf_dir": "conf",
            "default_data_file": "default_login_logs.csv",
            "default_rule_conf": "rule.conf",
            "data_file_encoding": "utf-8",
            "test_file_encoding": "utf-8"
        }
        self.config = {**default_config, **(config or {})}
        self.data_dir = self.config["data_dir"]
        self.conf_dir = self.config["conf_dir"]
        self.default_data_file_path = os.path.join(self.data_dir, self.config["default_data_file"])
        self.default_rule_conf_path = os.path.join(self.conf_dir, self.config["default_rule_conf"])

        # 创建目录并检查默认文件
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.conf_dir, exist_ok=True)

        logging.debug(f"初始化 ModelTrainer，配置：{self.config}")
        if not os.path.exists(self.default_data_file_path):
            logging.warning(f"默认数据文件 {self.default_data_file_path} 不存在。")
        if not os.path.exists(self.default_rule_conf_path):
            logging.warning(f"默认规则文件 {self.default_rule_conf_path} 不存在。")

        
    def __init_train_conf__(self, train_data_file=None, rule_conf=None):
        """
        初始化训练配置
        """
        # 检查文件路径是否有效
        logging.debug(f"初始化训练配置，输入文件：{train_data_file}, 规则配置：{rule_conf}")
        if train_data_file:
            if hasattr(train_data_file, "name"):  # 如果是上传的文件对象
                self.train_data_file = train_data_file.name
            else:
                self.train_data_file = train_data_file
            logging.info(f"使用上传的训练数据文件：{self.train_data_file}")
        else:
            if not os.path.exists(self.default_data_file_path):
                raise FileNotFoundError(f"默认数据文件 {self.default_data_file_path} 不存在，请上传文件或检查配置。")
            self.train_data_file = self.default_data_file_path
            logging.info(f"使用默认训练数据文件：{self.train_data_file}")

        # 加载规则配置
        if rule_conf is None:
            if not os.path.exists(self.default_rule_conf_path):
                raise FileNotFoundError(f"默认规则文件 {self.default_rule_conf_path} 不存在，请检查配置。")
            with open(self.default_rule_conf_path, "r", encoding="utf-8") as f:
                self.rule_conf = json.load(f)
            logging.info(f"加载默认规则配置：{self.default_rule_conf_path}")
        else:
            self.rule_conf = rule_conf
            logging.info("使用上传的规则配置。")
    
    def load_train_data(self, data_file_type, data_file):
        """
        加载训练数据文件
        """
        logging.debug(f"加载数据文件类型：{data_file_type}, 路径：{data_file}")
        if data_file_type not in ["test", "origin"]:
            raise ValueError(f"参数 data_file_type 的值必须为 'test' 或 'origin'，而不是 '{data_file_type}'")

        encoding = detect_encoding(data_file)
        logging.debug(f"检测到的数据文件编码：{encoding}")
        df = pd.read_csv(data_file, encoding=encoding)
        logging.info(f"{data_file_type} 数据文件加载完成，共 {len(df)} 行记录。")
        logging.debug(f"数据预览：\n{df.head()}")
        return df
    
    def preprocess_data(self, df):
        """
        数据清洗与特征提取
        """
        logging.info("开始数据清洗和特征提取...")
        df["登录时间"] = pd.to_datetime(df["登录时间"], errors="coerce")
        df["登录结果"] = df["登录结果"].apply(lambda x: 1 if x == "success" else 0)
        df["登录失败次数"] = df.groupby("用户ID")["登录结果"].transform(lambda x: (x == 0).sum())
        df["登录成功率"] = df.groupby("用户ID")["登录结果"].transform("mean")
        df["时间范围分钟"] = df.groupby("用户ID")["登录时间"].transform(lambda x: (x.max() - x.min()).total_seconds() / 60)
        df["登录失败次数"] = df.groupby("用户ID")["登录结果"].transform(lambda x: (x == 0).sum())
        df["每分钟失败比例"] = df["登录失败次数"] / df["时间范围分钟"]
       
        # 按用户ID和登录时间排序
        df = df.sort_values(by=["用户ID", "登录时间"])
        # 仅保留失败登录记录
        df["失败登录"] = (df["登录结果"] == 0).astype(int)
        
        # 计算时间间隔（秒）
        df["时间间隔"] = df.groupby("用户ID")["登录时间"].diff().dt.total_seconds()
        
        # 标记连续失败组（时间间隔大于 60 秒的视为新的组）
        df["失败组"] = (
            (df["失败登录"] == 1) & ((df["时间间隔"] > 60) | (df["时间间隔"].isna()))
        ).cumsum()
        
        # 统计每组失败次数
        df["连续失败次数"] = df.groupby(["用户ID", "失败组"])["失败登录"].transform("sum")
        
        # 添加连续失败3次特征
        df["连续失败3次"] = (df["连续失败次数"] >= 3).astype(int)

        encoder = LabelEncoder()
        df["用户编码"] = encoder.fit_transform(df["用户ID"])
        logging.info("数据清洗完成。")
        logging.info(f"清洗后的数据预览：\n{df.head()}")
        return df
    
    def build_model(self, df):
        """
        使用 Isolation Forest 构建异常检测模型
        """
        logging.info("开始训练异常检测模型...")

        X = df.drop(['异常类型'],axis=1) # 特征集，Drop掉标签相关字段
        y = df['异常类型']

        #将数据集进行80%（训练集）和20%（验证集）的分割
        from sklearn.model_selection import train_test_split #导入train_test_split工具
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.2, random_state=0)

        from sklearn.linear_model import LinearRegression # 导入线性回归算法模型
        model = LinearRegression() # 使用线性回归算法创建模型

        model.fit(X_train, y_train) # 用训练集数据，训练机器，拟合函数，确定参数

        y_pred = model.predict(X_test) #预测测试集的Y值
         
        df_ads_pred = X_test.copy() #测试集特征数据
        df_ads_pred['浏览量真值'] = y_test #测试集标签真值
        df_ads_pred['浏览量预测值'] = y_pred #测试集标签预测值
     
        logging.info("模型训练完成，生成异常检测结果。")
        return model, df_ads_pred
    
    def apply_anomaly_rules(self, df):
        anomaly_rules = AnomalyRules()
        #内网网段
        internal_subnets = ["192.168.0.0/16", "10.0.0.0/8", "172.16.0.0/12"]
        
        anomaly_rules.add_rule(
            "频繁登录失败", 
            lambda x: x["连续失败次数"] > 3,  # 失败间隔1分钟，连续失败3次
            "每分钟失败比例超过 0.1"
        )
        # anomaly_rules.add_rule(
        #     "高频登录", 
        #     lambda x: x["登录频率"] > 0.5, 
        #     "短时间内高频登录"
        # )
        anomaly_rules.add_rule(
            "非正常源地址",
            lambda x: ~x["登录地址"].apply(lambda ip: ip_in_subnet(ip, internal_subnets)),
            "登录地址属于可疑网段"
        )
        anomaly_rules.add_rule(
            "未知用户登录", 
            lambda x: x["用户ID"] == "unknown_user", 
            "未知用户尝试登录"
        )
        
        df = anomaly_rules.apply_rules(df)
        return df
     
    # def combine_anomaly_results(self, df, user_stats):
    #     """
    #     合并模型和规则的异常检测结果
    #     """
    #     # 合并必要的列，包括登录失败次数、登录成功率、平均登录间隔等
    #     df = df.merge(user_stats[["用户ID", "登录失败次数", "登录成功率", "平均登录间隔", "模型异常"]], on="用户ID", how="left")
        
    #     df["是否异常"] = df.apply(
    #         lambda row: 1 if row["模型异常"] == -1 or row["规则异常"] == 1 else 0, axis=1
    #     )
    #     logging.info("异常检测结果合并完成")
    #     return df

    def visualize_anomalies(self, df):
        """
        绘制按异常类型分类的统计结果
        """
        # 获取总记录数
        total_count = len(df)

        # 统计异常类型数量
        anomaly_counts = df["异常类型"].value_counts()

        # 统计正常登录数量
        normal_count = len(df[df["是否异常"] == 0])

        # 合并正常登录和异常类型统计
        all_counts = pd.concat([anomaly_counts, pd.Series({"正常登录": normal_count})])
        all_ratios = all_counts / total_count * 100

        # 绘制饼图
        plt.figure(figsize=(8, 8))
        plt.pie(
            all_counts,
            labels=[f"{label} ({ratio:.1f}%)" for label, ratio in zip(all_counts.index, all_ratios)],
            autopct="%1.1f%%",
            startangle=90,
            colors=plt.cm.tab20.colors[:len(all_counts)],  # 使用预定义颜色
            textprops={"fontproperties": my_font}  # 设置中文字体
        )
        plt.title("登录行为分布", fontproperties=my_font)

        # 保存图表
        plot_file_path = os.path.join(self.data_dir, "anomalies_plot.png")
        plt.savefig(plot_file_path, format="png")

        # 显示图表（可选）
        plt.show()

        # 保存异常记录到文件
        anomalies = df[df["是否异常"] == 1]
        anomalies_file_path = os.path.join(self.data_dir, "anomalies.csv")
        anomalies.to_csv(anomalies_file_path, index=False, encoding=self.config["data_file_encoding"])

        logging.info("\n检测到的异常登录记录：")
        logging.info(anomalies[["用户ID", "登录时间", "登录地址", "登录资源", "登录失败次数", "登录成功率", "异常类型"]].head(100))

        return anomalies_file_path, anomalies, plot_file_path

       

    # def evaluate_model(self, model, test_user_stats):
    #     """
    #     使用测试数据评估模型
    #     """
    #     features = ["登录失败次数", "登录成功率"]
    #     X_test = test_user_stats[features]

    #     # 使用模型进行预测
    #     test_user_stats["模型预测"] = model.predict(X_test)
    #     test_user_stats["模型异常"] = test_user_stats["模型预测"]  # 将模型预测结果映射为模型异常列

    #     # 计算准确率、召回率、F1等指标
    #     y_true = test_user_stats["模型异常"]
    #     y_pred = test_user_stats["模型预测"]

    #     # 输出分类报告
    #     report = classification_report(y_true, y_pred, target_names=["正常", "异常"])
    #     cm = confusion_matrix(y_true, y_pred)
        
    #     logging.info(f"模型评估报告：\n{report}")
    #     logging.info(f"混淆矩阵：\n{cm}")
    #     return report, cm
    
    def optimize_model(self, X_train, y_train, model):
        """
        使用网格搜索对模型进行优化
        """
        param_grid = {
            "contamination": [0.05, 0.1, 0.15, 0.2],  # 异常比例
            "n_estimators": [50, 100, 200],           # 决策树数量
            "max_samples": [0.8, 1.0]                  # 训练样本比例
        }

        grid_search = GridSearchCV(model, param_grid, cv=5, scoring="accuracy", verbose=2)
        grid_search.fit(X_train, y_train)

        # 输出最优参数
        logging.info(f"最优参数：{grid_search.best_params_}")
        logging.info(f"最佳模型得分：{grid_search.best_score_}")

        return grid_search.best_estimator_
 
    def train(self,uploaded_file=None, rules_input=None):
        
        logging.info("开始训练流程...")
        #0. 初始化数据文件和规则
        self.__init_train_conf__(uploaded_file, rules_input)
        #1.数据收集file-原始审计数据-选择1天的行为审计数据-格式化为csv文件-外部程序实现 access_login_data.csv
        #2.数据可视化查看数据展示数据关系-略
        #3.数据清洗- 根据模型定义问题处理
           #3.1 根据编码读取数据
        df = self.load_train_data(data_file_type="origin", data_file=self.train_data_file)
           #3.2 数据清洗和特征提取
        pre_df = self.preprocess_data(df)
        #4. 构建特征集和标签集（监督学习需要标签集）
        tag_df = self.apply_anomaly_rules(pre_df)
        
        anomalies_file_path, anomalies, plot_file_path = self.visualize_anomalies(tag_df)
        
        summar_anomalie = anomalies[["用户ID", "登录地址", "登录失败次数", "登录成功率", "平均登录间隔", "异常类型"]].head(100)
        
        #5. 选择算法并建立模型、训练模型-返回模型和预测结果
        model, user_stats = self.build_model(tag_df)
        #6. 模型优化
           #6.1 使用规则检测异常
        # rule_pd = self.apply_anomaly_rules(user_stats)
           #6.2 合并模型和规则结果
        #com_df = self.combine_anomaly_results(pre_df, rule_pd)
           #6.3 规则可视化并保存到文件
        
    
        #7.模型保存
        model_file = "anomaly_detection_model.pkl"
        import joblib
        joblib.dump(model, model_file)
        logging.info(f"模型已保存到 {model_file}")
        
        # #8. 生成测试集 
        # generate_test_data(file_path="user_login_test_data.csv")
        # #9. 加载测试数据
        # test_df = self.load_train_data(data_file_type="test",data_file="user_login_test_data.csv")
        # #10. 使用训练好的模型对测试数据生成预测结果
        # features = ["登录失败次数", "登录成功率", "平均登录间隔", "活跃小时数"]
        # X_test = test_df[features]
        # test_df["模型异常"] = model.predict(X_test)  # 使用模型生成预测结果
        
        # self.evaluate_model(model, test_df)
        
        # #11. 优化模型
        # features = ["登录失败次数", "登录成功率", "平均登录间隔", "活跃小时数"]
        # X_train = user_stats[features]
        # y_train = user_stats["模型异常"]
        
        # optimized_model = self.optimize_model(X_train, y_train, model)
        # # 保存优化后的模型
        # optimized_model_file = "optimized_anomaly_detection_model.pkl"
        # joblib.dump(optimized_model, optimized_model_file)
        # logging.info(f"优化后的模型已保存到 {optimized_model_file}")
        
        return anomalies_file_path, summar_anomalie, plot_file_path, model_file

In [1]:


# ------------------------------
# 主程序
# ------------------------------
if __name__ == "__main__":
    logging.info("启动训练流程...")
    try:
        model = ModelTrainer()
        anomalies_file_path, summar_anomalie, plot_file_path, model_file = model.train()
        logging.info(f"训练完成！\n异常文件路径: {anomalies_file_path}\n图表路径: {plot_file_path}\n模型路径: {model_file}")
    except Exception as e:
        logging.error(f"训练过程中发生错误: {str(e)}")


ModuleNotFoundError: No module named 'utils'